In [10]:
import datasets
import transformers
import rouge

### Loading Data

In [11]:
train_data =       datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
validation_data =  datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation")
test_data =        datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")

Reusing dataset cnn_dailymail (/Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Reusing dataset cnn_dailymail (/Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Reusing dataset cnn_dailymail (/Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


### Sample of the data

In [12]:
def show_examples(dataset, num_samples=3, seed=42):
    samples = dataset.shuffle(seed=seed).select(range(num_samples))
        
    for idx, sample in enumerate(samples):
        display(f'sample {idx}: {sample["article"]} \n')
        display(f'highlight {idx}: {sample["highlights"]} \n')
        display(f'id: {sample["id"]}')
        display('-------')
        
def get_samples(dataset, num_samples=10):
    return dataset.shuffle(seed=1).select(range(num_samples))

def get_random_sample(dataset):
    sample = dataset.shuffle(seed=1).select(range(1)) 
    return [sample["article"][0], sample["highlights"][0]]

In [13]:
# get_random_sample(train_data)

### Tokenizer

In [14]:
batch_size=4 # change to 16 for full training
encoder_max_length=512
decoder_max_length=128

tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")
# tokenizer = transformers.DistilBertTokenizerFast()
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

def tokenize_data_to_model_input(batch):
    inputs  = tokenizer(batch["article"], padding="max_length", 
                       truncation=True, max_length=encoder_max_length)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["highlights"], 
                           padding="max_length", truncation=True, max_length=decoder_max_length)
    
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    
    batch["decoder_input_ids"] = labels.input_ids
    batch["decoder_attention_mask"] = labels.attention_mask
    batch["labels"] = labels["input_ids"]
    
    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]
    
    return batch

# For now subsample is being used
percentile = 0.0005
amount = round(len(train_data) * percentile)
print(amount)
train_data = train_data.select(range(amount))

train_data = train_data.map(
    tokenize_data_to_model_input, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
amount = round(len(validation_data) * percentile)
validation_data = validation_data.select(range(amount))

validation_data = validation_data.map(
    tokenize_data_to_model_input, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)
validation_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


test_data = test_data.select(range(128))
# test_data = test_data.map(
#     tokenize_data_to_model_input, 
#     batched=True, 
#     batch_size=batch_size, 
#     remove_columns=["article", "highlights", "id"]
# )
# test_data.set_format(type="torch")
    

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/jeroen/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0.dev0",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/jeroen/.cache/huggin

144


100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 246.31ba/s]


### Encoder - Decoder

In [15]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig, EncoderDecoderModel, EncoderDecoderConfig

# bert2bert = transformers.AutoModelForSeq2SeqLM.from_pretrained("bert-base-uncased", output_loading_info=True)



# config_encoder = DistilBertConfig.from_pretrained('distilbert-base-uncased')
# config_decoder = DistilBertConfig.from_pretrained('distilbert-base-uncased')

# config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)

# distilbert2distilbert = EncoderDecoderModel(config=config)

bert2bert = transformers.EncoderDecoderModel.from_encoder_decoder_pretrained("distilbert-base-uncased", "distilbert-base-uncased")


loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/jeroen/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0.dev0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /Users/jero

In [16]:
# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 142
bert2bert.config.min_length = 56
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

### Evaluation Metric

In [17]:
from rouge import Rouge 

rouge_scorer = Rouge()

def compute_evaluation_metric(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    score = rouge_scorer.get_scores(label_str, pred_str)
    f = score[0]["rouge-2"]["f"]
     
    return {
        "rouge2_fmeasure": f
    }

### Training

In [19]:
from typing_extensions import Protocol, runtime_checkable
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 2
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy = "epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_evaluation_metric,
    train_dataset=train_data,
    eval_dataset=validation_data,
)


trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 144
  Num Epochs = 2
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 144


Epoch,Training Loss,Validation Loss,Rouge2 Fmeasure
1,No log,nan,0.000000
2,No log,nan,0.000000


***** Running Evaluation *****
  Num examples = 7
  Batch size = 2
***** Running Evaluation *****
  Num examples = 7
  Batch size = 2


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=144, training_loss=0.0, metrics={'train_runtime': 349.5842, 'train_samples_per_second': 0.824, 'train_steps_per_second': 0.412, 'total_flos': 109083781693440.0, 'train_loss': 0.0, 'epoch': 2.0})

In [24]:
dir_name = 'distilbert-v1'
trainer.save_model(dir_name)

Saving model checkpoint to distilbert-v1
Configuration saved in distilbert-v1/config.json
Model weights saved in distilbert-v1/pytorch_model.bin
tokenizer config file saved in distilbert-v1/tokenizer_config.json
Special tokens file saved in distilbert-v1/special_tokens_map.json


### Evaluation

In [27]:
# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

In [29]:
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")
test_data = test_data.select(range(32))
model = transformers.EncoderDecoderModel.from_pretrained(dir_name)

results = test_data.map(generate_summary, batched=True, batch_size=batch_size)


Reusing dataset cnn_dailymail (/Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
loading configuration file distilbert-v1/config.json
Model config EncoderDecoderConfig {
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "distilbert-base-uncased",
    "activation": "gelu",
    "add_cross_attention": true,
    "architectures": [
      "DistilBertForMaskedLM"
    ],
    "attention_dropout": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "dim": 768,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "hidden_dim": 3072,
    "id2label":

## Displaying Results

### Util function

In [ ]:
# Credits tot Mauro Di Pietro
# https://towardsdatascience.com/text-summarization-with-nlp-textrank-vs-seq2seq-vs-bart-474943efeb09

import re
import difflib
import nltk
# nltk.download()

'''
Find the matching substrings in 2 strings.
:parameter
    :param a: string - raw text
    :param b: string - raw text
:return
    2 lists used in to display matches
'''
def utils_split_sentences(a, b):
    ## find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]
    
    ## difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)
    
    ## work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        ### a
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        ### b
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
    
    return lst_a, lst_b


'''
Highlights the matched strings in text.
:parameter
    :param a: string - raw text
    :param b: string - raw text
    :param both: bool - search a in b and, if True, viceversa
    :param sentences: bool - if False matches single words
:return
    text html, it can be visualized on notebook with display(HTML(text))
'''
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()       
    
    ## highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)
    
    ## highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b) 
    second_text = ' '.join(second_text)
    
    ## concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br><br>'+ second_text
    return final_text

## Results

In [ ]:
from IPython.core.display import display, HTML
from rouge import Rouge 

rouge_new = Rouge()

for i in range(10):
    article = results["article"][i]
    highlight = results["highlights"][i]
    prediction = results["pred"][i]
    score = rouge_new.get_scores(highlight, prediction)
    
    rouge_1_f = score[0]["rouge-1"]["f"] * 100
    rouge_2_f = score[0]["rouge-2"]["f"] * 100
    s = f"rouge-1: {rouge_1_f}, rouge-2:  {rouge_2_f}" 
    
#     match_article_prediction = display_string_matching(article, prediction, 
#                                                        both=True, sentences=False,
#                                                       titles=["Article", "Predicted Summary"])
    
    match_summary = display_string_matching(highlight, prediction, both=True, 
                                    sentences=False, 
                                    titles=["Real Summary", f"Predicted Summary ({s})"])
    
    display(HTML(match_summary))
    print("---")